In [2]:
import numpy as np
import os
import pathlib
import cv2
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential

In [4]:
# Define constants
IMAGE_SHAPE = (200, 200)  # Resize all images to this shape for MobileNetV2
BATCH_SIZE = 100
DATA_DIR = 'C:/Users/Indhu/Downloads/MRI brain/'  # Update with your dataset path

In [24]:
# Load data
data_dir = pathlib.Path(DATA_DIR)
categories = os.listdir(DATA_DIR)
mri_images_dict = {}
mri_labels_dict = {}
X, y = [], []
for index, category in enumerate(categories):
   mri_images_dict[category] = list(data_dir.glob(category + '/*'))
   mri_labels_dict[category] = index

for category_name, images in mri_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, IMAGE_SHAPE)
        X.append(resized_img)
        y.append(mri_labels_dict[category_name])

X = np.array(X) / 255.0  # Normalize images
y = np.array(y)

In [38]:
# Count images per class
class_counts = {category: len(images) for category, images in mri_images_dict.items()}

# Print counts for each class
for category, count in class_counts.items():
    print(f"Class '{category}': {count} images")


Class 'glioma': 300 images
Class 'meningioma': 306 images
Class 'notumor': 405 images
Class 'pituitary': 300 images


In [28]:
print(f"Loaded {len(X)} images with shape {X.shape} and {len(y)} labels.")
print(f"X shape: {X.shape}, y shape: {y.shape}")

Loaded 1311 images with shape (1311, 200, 200, 3) and 1311 labels.
X shape: (1311, 200, 200, 3), y shape: (1311,)


In [8]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
# Calculate the percentages
train_percentage = len(X_train) / len(X) * 100
test_percentage = len(X_test) / len(X) * 100

# Print the results
print(f"Training data: {len(X_train)} {X_train.shape} samples ({train_percentage:.2f}%)")
print(f"Testing data: {len(X_test)} {X_test.shape} samples ({test_percentage:.2f}%)")

Training data: 1048 (1048, 200, 200, 3) samples (79.94%)
Testing data: 263 (263, 200, 200, 3) samples (20.06%)


In [10]:
base_model_mobilenet = MobileNetV2(input_shape=IMAGE_SHAPE + (3,), include_top=False, weights='imagenet')
base_model_mobilenet.trainable = False  # Freeze the convolutional base

feature_extractor_mobilenet = Sequential([
    base_model_mobilenet,
    GlobalAveragePooling2D()
])

C:\Users\Indhu\AppData\Local\Temp\ipykernel_7644\327665529.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model_mobilenet = MobileNetV2(input_shape=IMAGE_SHAPE + (3,), include_top=False, weights='imagenet')


In [58]:
# Extract features using MobileNetV2
import time
start_trainfeatures_time = time.time()
X_train_features = feature_extractor_mobilenet.predict(X_train, batch_size=BATCH_SIZE, verbose=1)  
end_trainfeatures_time = time.time()
start_testfeatures_time = time.time()
X_test_features = feature_extractor_mobilenet.predict(X_test, batch_size=BATCH_SIZE, verbose=1) 
end_testfeatures_time = time.time()

11/11 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step


In [62]:
# Calculate time taken for training and testing
train_time = end_trainfeatures_time - start_trainfeatures_time
test_time = end_testfeatures_time - start_testfeatures_time
print(f"Training time: {train_time} seconds")
print(f"Testing time: {test_time} seconds")

Training time: 21.756541967391968 seconds
Testing time: 5.844156265258789 seconds


In [14]:
# Initialize classifiers
classifiers = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=200, random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
    "Support Vector Machine": SVC(kernel='rbf', probability=True, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Naive Bayes": GaussianNB()
}

In [54]:
# Evaluate classifiers using test set
for model_name, model in classifiers.items():
    print(f"Evaluating {model_name}...")
    start_train_time = time.time()
    model.fit(X_train_features, y_train)
    end_train_time = time.time()
    start_test_time = time.time() 
    y_pred = model.predict(X_test_features)
    end_test_time = time.time()
    print(f"{model_name} Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred))

Evaluating Random Forest...
Random Forest Accuracy: 0.8479
              precision    recall  f1-score   support

           0       0.83      0.79      0.81        61
           1       0.76      0.58      0.66        60
           2       0.94      0.99      0.96        80
           3       0.81      0.98      0.89        62

    accuracy                           0.85       263
   macro avg       0.84      0.84      0.83       263
weighted avg       0.84      0.85      0.84       263

Evaluating Logistic Regression...
Logistic Regression Accuracy: 0.8821
              precision    recall  f1-score   support

           0       0.87      0.89      0.88        61
           1       0.78      0.72      0.75        60
           2       0.94      0.97      0.96        80
           3       0.90      0.92      0.91        62

    accuracy                           0.88       263
   macro avg       0.87      0.87      0.87       263
weighted avg       0.88      0.88      0.88       263



In [56]:
# Calculate time taken for training and testing
train_time = end_train_time - start_train_time
test_time = end_test_time - start_test_time
print(f"Training time: {train_time} seconds")
print(f"Testing time: {test_time} seconds")

Training time: 0.015629053115844727 seconds
Testing time: 0.031249046325683594 seconds


In [72]:
# Evaluate classifiers using k-fold cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for clf_name, clf in classifiers.items():
    print(f"K-Fold Cross Validation for {clf_name}...")
    precision_scores = []
    recall_scores = []
    f1_scores = []
    accuracy_scores = []

    for train_index, val_index in kf.split(X_train_features, y_train):
        X_train_fold, X_val_fold = X_train_features[train_index], X_train_features[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        clf.fit(X_train_fold, y_train_fold)
        y_val_pred = clf.predict(X_val_fold)

        accuracy_scores.append(accuracy_score(y_val_fold, y_val_pred))
        precision_scores.append(precision_score(y_val_fold, y_val_pred, average='weighted'))
        recall_scores.append(recall_score(y_val_fold, y_val_pred, average='weighted'))
        f1_scores.append(f1_score(y_val_fold, y_val_pred, average='weighted'))

    print(f"Average Accuracy: {np.mean(accuracy_scores):.4f}")
    print(f"Average Precision: {np.mean(precision_scores):.4f}")
    print(f"Average Recall: {np.mean(recall_scores):.4f}")
    print(f"Average F1 Score: {np.mean(f1_scores):.4f}\n")

K-Fold Cross Validation for Random Forest...
Average Accuracy: 0.8597
Average Precision: 0.8600
Average Recall: 0.8597
Average F1 Score: 0.8547

K-Fold Cross Validation for Logistic Regression...
Average Accuracy: 0.8826
Average Precision: 0.8828
Average Recall: 0.8826
Average F1 Score: 0.8805

K-Fold Cross Validation for K-Nearest Neighbors...
Average Accuracy: 0.8129
Average Precision: 0.8146
Average Recall: 0.8129
Average F1 Score: 0.8013

K-Fold Cross Validation for Support Vector Machine...
Average Accuracy: 0.8864
Average Precision: 0.8872
Average Recall: 0.8864
Average F1 Score: 0.8841

K-Fold Cross Validation for Decision Tree...
Average Accuracy: 0.6660
Average Precision: 0.6692
Average Recall: 0.6660
Average F1 Score: 0.6659

K-Fold Cross Validation for Naive Bayes...
Average Accuracy: 0.7958
Average Precision: 0.7911
Average Recall: 0.7958
Average F1 Score: 0.7911

